# Sampling correction for large choice sets

1. Replicate synthetic data from Guevara & Ben-Akiva 2013
2. Do MNL with and without sampling correction
3. Check whether parameter estimates deviate from true values
4. Extend to Mixed Logit

## 1. Generate synthetic data set

- N = 1000 observations
- J = 1000 alternatives for all observations (C_n = C)
- X = single attribute distributed Uniform(-2,1) for the first 500 alternatives and Uniform(-1,2) for the second half
- beta = generic linear taste coefficient, distributed Normal(mu=1.5, sigma=0.8) across the 1000 observations
- systematic utility = beta * X
- epsilon = error term distributed ExtremeValue(0,1)
- random utility = beta * X + epsilon

Utility of alternative i for agent n:
$$ U_{in} = V_{in} + \varepsilon_{in} = \beta_n x_{i} + \varepsilon_{in} $$

Probability that agent n will choose alternative i:
$$ L_n(i \mid \beta_n, x_n,C_n) = \frac {e^{V_{in}}} {\sum_{j \epsilon C_n} e^{V_{jn}}} $$

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Generate attribute x for each of J alternatives

# Set a seed for reproducibility
np.random.seed(12)

# Start with J << 1000 to speed up runtimes

J = 50  # alternatives

Xa = 3 * np.random.rand(J/2) - 2  # uniform distribution over [-2, 1]
Xb = 3 * np.random.rand(J/2) - 1  # uniform distribution over [-1, 2]

X = np.concatenate((Xa, Xb))

print len(X)
print X[:5]

50
[-1.53751147  0.22014909 -1.21005495 -0.39878182 -1.95627511]


In [4]:
# Generate taste coefficient beta for each of N agents 

# For regular MNL, i think we need to use a single value, instead of a 
# distribution as Guevara & Ben-Akiva used for the mixture model

N = 1000  # agents/observations

beta = np.zeros(1000) + 1.5
# beta = 0.8 * np.random.randn(N) + 1.5

print len(beta)
print beta[:5]

1000
[ 1.5  1.5  1.5  1.5  1.5]


In [ ]:
print pd.DataFrame(beta).describe()

In [5]:
# Generate probability matrix for N agents choosing among J alternatives

def probs(n):
    ''' 
    Return list of J probabilities for agent n
    '''
    b = beta[n]
    exps = [np.exp(b*x) for x in X]
    sum_exps = np.sum(exps)
    return [exp/sum_exps for exp in exps]

P = np.array([probs(n) for n in range(N)])
    
print P.shape

(1000, 50)


In [6]:
# Check that each row sums to 1

print np.sum(P, axis=1)[:10]

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [7]:
# Simulate a choice from J alternatives for each of N agents

C = [np.random.choice(range(J), p=p) for p in P]

print len(C)
print C[:10]

1000
[12, 41, 37, 5, 30, 27, 8, 35, 33, 6]


#### Now we have data:

- N agents/observations with true taste coefficients in array "beta"
- J alternatives with single attributes in array "X"
- N choice outcomes in array "C"

## 2. Estimate beta without sampling, using PyLogit MNL

In [8]:
import pylogit
from collections import OrderedDict

In [9]:
# Set up an estimation dataset in long format

d = [[n, i, int(C[n]==i), X[i]] for n in range(N) for i in range(J)]

print len(d)

50000


In [10]:
df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'choice', 'x'])

print df.describe()

             obs_id        alt_id        choice             x
count  50000.000000  50000.000000  50000.000000  50000.000000
mean     499.500000     24.500000      0.020000      0.014570
std      288.677877     14.431014      0.140001      1.116965
min        0.000000      0.000000      0.000000     -1.993222
25%      249.750000     12.000000      0.000000     -0.894495
50%      499.500000     24.500000      0.000000      0.220035
75%      749.250000     37.000000      0.000000      0.832675
max      999.000000     49.000000      1.000000      1.985414


In [11]:
# Set up model spec

spec = OrderedDict([
        ('x', [range(J)])
    ])

labels = OrderedDict([
        ('x', ['beta_x'])
    ])

In [20]:
%%time
m = pylogit.create_choice_model(data = df, 
                                alt_id_col = 'alt_id', 
                                obs_id_col = 'obs_id', 
                                choice_col = 'choice', 
                                specification = spec, 
                                model_type = "MNL", 
                                names = labels)

m.fit_mle(init_vals = np.array([0]))
print m.get_statsmodels_summary()

Log-likelihood at zero: -3,912.0230
Initial Log-likelihood: -3,912.0230
Estimation Time: 0.06 seconds.
Final log-likelihood: -3,065.1983
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:                1,000
Model:             Multinomial Logit Model   Df Residuals:                      999
Method:                                MLE   Df Model:                            1
Date:                     Sun, 20 Nov 2016   Pseudo R-squ.:                   0.216
Time:                             10:57:48   Pseudo R-bar-squ.:               0.216
converged:                            True   Log-Likelihood:             -3,065.198
                                             LL-Null:                    -3,912.023
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
beta_x         1.5324      0.046 

## 3. Try with UrbanSim MNL instead of PyLogit

Model class: https://github.com/UDST/urbansim/blob/master/urbansim/models/dcm.py

Estimation algorithms: https://github.com/UDST/urbansim/blob/master/urbansim/urbanchoice/mnl.py

In [13]:
from urbansim.models import MNLDiscreteChoiceModel

In [43]:
# Choosers should be a DataFrame of characteristics, with index as identifier

d = [[n, C[n]] for n in range(N)]

choosers = pd.DataFrame(d, columns=['id', 'choice']).set_index('id')

print len(choosers)

1000


In [44]:
# Alternatives should be a DataFrame of characteristics, with index as identifier

d = [[i, X[i]] for i in range(J)]

alts = pd.DataFrame(d, columns=['id', 'x']).set_index('id')

print len(alts)

50


In [61]:
# It seems like this implementation *requires* us to sample the alternatives, 
# so here i'm estimating the model with J-1 alts

m = MNLDiscreteChoiceModel(model_expression = 'x',
                           sample_size = J-1)

m.fit(choosers = choosers,
      alternatives = alts,
      current_choice = 'choice')

m.report_fit()

Null Log-liklihood: -3891.820
Log-liklihood at convergence: -3072.798
Log-liklihood Ratio: 0.210

+-----------+-------------+------------+---------+
| Component | Coefficient | Std. Error | T-Score |
+-----------+-------------+------------+---------+
| x         |    1.528    |   0.022    |  69.281 |
+-----------+-------------+------------+---------+


## 4. MNL, sampling alternatives without correction

In [ ]:
# In the estimation dataset, for each observation include a row for the
# chosen alternative, plus K-1 other alternatives sampled randomly
# without replacement, where K < J.

In [65]:
K = 10

def alts(obs_id):
    """
    Sample alternatives for observation `obs_id`. Expects `J` total
    alts, `K` sampled alts, list `C` of choice outcomes. Returns list 
    of K alt id's including the chosen one.
    """
    chosen = C[obs_id]  # id of chosen alternative
    unchosen = [i for i in range(J) if chosen != i]  # id's of J-1 unchosen alts
    sample_unchosen = np.random.choice(unchosen, size=K-1, replace=False)
    return [chosen] + sample_unchosen.tolist()
    
print alts(0)

[12, 46, 32, 47, 1, 43, 42, 11, 35, 9]


In [66]:
# Set up the estimation dataset

d = [[n, i, int(C[n]==i), X[i]] for n in range(N) for i in alts(n)]

print len(d)

10000


In [67]:
df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'choice', 'x'])

print df.head(), '\n'
print df.describe()

   obs_id  alt_id  choice         x
0       0      12       1  0.832675
1       0       4       0 -1.956275
2       0      11       0 -0.181750
3       0      35       0  1.850941
4       0      30       0  1.107867 

             obs_id        alt_id        choice             x
count  10000.000000  10000.000000  10000.000000  10000.000000
mean     499.500000     25.173500      0.100000      0.118479
std      288.689425     14.329589      0.300015      1.154237
min        0.000000      0.000000      0.000000     -1.993222
25%      249.750000     13.000000      0.000000     -0.543868
50%      499.500000     26.000000      0.000000      0.257751
75%      749.250000     37.000000      0.000000      0.873746
max      999.000000     49.000000      1.000000      1.985414


In [68]:
# Same model spec as before

spec = OrderedDict([
        ('x', [range(J)])
    ])

labels = OrderedDict([
        ('x', ['beta_x'])
    ])

In [69]:
%%time
m = pylogit.create_choice_model(data = df, 
                                alt_id_col = 'alt_id', 
                                obs_id_col = 'obs_id', 
                                choice_col = 'choice', 
                                specification = spec, 
                                model_type = "MNL", 
                                names = labels)

m.fit_mle(init_vals = np.array([0]))
print m.get_statsmodels_summary()

Log-likelihood at zero: -2,302.5851
Initial Log-likelihood: -2,302.5851
Estimation Time: 0.02 seconds.
Final log-likelihood: -1,566.8267
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      choice   No. Observations:                1,000
Model:             Multinomial Logit Model   Df Residuals:                      999
Method:                                MLE   Df Model:                            1
Date:                     Sun, 20 Nov 2016   Pseudo R-squ.:                   0.320
Time:                             12:00:38   Pseudo R-bar-squ.:               0.319
converged:                            True   Log-Likelihood:             -1,566.827
                                             LL-Null:                    -2,302.585
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
beta_x         1.5100      0.051 

### Run repeatedly and calculate average beta

In [78]:
def estimate_beta():
    d = [[n, i, int(C[n]==i), X[i]] for n in range(N) for i in alts(n)]
    df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'choice', 'x'])
    m = pylogit.create_choice_model(df, 'alt_id', 'obs_id', 'choice', spec, 'MNL', names=labels)
    m.fit_mle(init_vals = np.array([0]))
    return m.params.beta_x

In [81]:
%%capture

beta = []
for i in range(50):
    beta.append(estimate_beta())

In [82]:
print pd.Series(beta).describe()

count    50.000000
mean      1.522650
std       0.023406
min       1.480828
25%       1.506538
50%       1.523193
75%       1.536155
max       1.579116
dtype: float64


Appears biased downward from true coef of 1.532

In [ ]:
# To do 
# - look through PyLogit and LCCM code
# - in many-alternative scenarios, attirbutes of the alternatives will 
#   usually be in a separate data table - what helper functions do we need?

